In [1]:
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import copy, time
import random
import pickle

import mlrfit as mf

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1001)
random.seed(1001)

#  Matrix definition

In [3]:
rank = 20
mtype = "dgt"

m = 3500
n = 3200

A = mf.dgt_matrix(m, n, d=3, k=5)

In [4]:
A.min(), A.mean(), A.max(), 1.*(A>1e-6).sum()/A.size

(1.992613960798277e-30,
 0.031070391228852242,
 0.9998269468836632,
 0.6146209821428571)

In [5]:
max_num_levels = int(np.ceil(np.log2(min(m,n)))+1)

# LR model

In [6]:
hpart = mf.random_hpartition(m, n, num_levels=1, perm=False)
B1, C1 = mf.single_level_factor_fit(A, np.array([rank]), hpart, level=0)[:2]
lr_losses = [mf.rel_diff(B1 @C1.T, den=A)]
print(lr_losses[-1])

0.5087598386871721


# Factor fit 
### given (random) hierarchy

#### Define random hierarchy 

In [7]:
pi_rows, pi_cols = np.random.permutation(m), np.random.permutation(n)
hpart = {'rows':{'pi':pi_rows, 'lk':[]}, 'cols':{'pi':pi_cols, 'lk':[]}} 
level_list = range(4)
for level in level_list:
    if 2**level+1 <= m and 2**level+1 <= n: 
        hpart['rows']['lk'] += [ np.linspace(0, m, 2**level+1, endpoint=True, dtype=int)]
        hpart['cols']['lk'] += [ np.linspace(0, n, 2**level+1, endpoint=True, dtype=int)]
    else:
        hpart['rows']['lk'] += [ np.linspace(0, m, min(m,n)+1, endpoint=True, dtype=int)]
        hpart['cols']['lk'] += [ np.linspace(0, n, min(m,n)+1, endpoint=True, dtype=int)]

In [8]:
mf.hpart_info_print(hpart)

level=0,  1
    avg_row_bl_size=3500.0, avg_col_bl_size=3200.0
level=1,  2
    avg_row_bl_size=1750.0, avg_col_bl_size=1600.0
level=2,  4
    avg_row_bl_size=875.0, avg_col_bl_size=800.0
level=3,  8
    avg_row_bl_size=437.5, avg_col_bl_size=400.0


#### Rank allocation `ranks`

In [9]:
ranks = mf.uniform_ranks(rank, len(hpart['rows']['lk'])) 
print(f"{ranks=}")

ranks=array([5, 5, 5, 5])


#### Factor fit for given `hpart` and `ranks`

In [10]:
hat_A = mf.MLRMatrix()
ff_losses = hat_A.factor_fit(A, ranks, hpart, freq=5, printing=True)

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/mlrfit-0.0.1-py3.9.egg/mlrfit/utils.py:433: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, A))
  hat_A_except_level[r1:r2, c1:c2] += np.dot(B_level[r1:r2], C_level[c1:c2].T)


itr=0, 0.7111209374759613, [5 5 5 5]


# Full fit
### rank allocation + spectral partitioning + greedy refinement

#### Initial rank allocation `ranks`

In [11]:
ranks = mf.uniform_ranks(rank, max_num_levels) 
print(f"{ranks=}")

ranks=array([2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1])


#### Find `hpart` for initial rank allocation `ranks`

In [12]:
hat_A = mf.MLRMatrix()
_, _ = hat_A.hpartition_topdown(A, ranks)

* level=0, losses[0]=0.907, losses[-1]=0.907, len(losses)=2, [2]


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


* level=1, losses[0]=0.801, losses[-1]=0.797, len(losses)=2, [2 2]
* level=2, losses[0]=0.687, losses[-1]=0.682, len(losses)=2, [2 2 2]
* level=3, losses[0]=0.574, losses[-1]=0.569, len(losses)=2, [2 2 2 2]
* level=4, losses[0]=0.496, losses[-1]=0.491, len(losses)=2, [2 2 2 2 2]
* level=5, losses[0]=0.454, losses[-1]=0.450, len(losses)=2, [2 2 2 2 2 2]
* level=6, losses[0]=0.436, losses[-1]=0.434, len(losses)=2, [2 2 2 2 2 2 2]
* level=7, losses[0]=0.429, losses[-1]=0.428, len(losses)=2, [2 2 2 2 2 2 2 1]
* level=8, losses[0]=0.424, losses[-1]=0.423, len(losses)=2, [2 2 2 2 2 2 2 1 1]
* level=9, losses[0]=0.420, losses[-1]=0.420, len(losses)=2, [2 2 2 2 2 2 2 1 1 1]
* level=10, losses[0]=0.418, losses[-1]=0.417, len(losses)=2, [2 2 2 2 2 2 2 1 1 1 1]
* level=11, losses[0]=0.416, losses[-1]=0.415, len(losses)=2, [2 2 2 2 2 2 2 1 1 1 1 1]
* level=12, losses[0]=0.414, losses[-1]=0.414, len(losses)=2, [2 2 2 2 2 2 2 1 1 1 1 1 1]
level=12, loss=0.4135701178285843


In [13]:
mf.hpart_info_print(hat_A.hpart)

level=0,  1
    avg_row_bl_size=3500.0, avg_col_bl_size=3200.0
level=1,  2
    avg_row_bl_size=1750.0, avg_col_bl_size=1600.0
level=2,  4
    avg_row_bl_size=875.0, avg_col_bl_size=800.0
level=3,  8
    avg_row_bl_size=437.5, avg_col_bl_size=400.0
level=4,  16
    avg_row_bl_size=218.8, avg_col_bl_size=200.0
level=5,  32
    avg_row_bl_size=109.4, avg_col_bl_size=100.0
level=6,  64
    avg_row_bl_size=54.7, avg_col_bl_size=50.0
level=7,  128
    avg_row_bl_size=27.3, avg_col_bl_size=25.0
level=8,  256
    avg_row_bl_size=13.7, avg_col_bl_size=12.5
level=9,  512
    avg_row_bl_size=6.8, avg_col_bl_size=6.2
level=10,  1024
    avg_row_bl_size=3.4, avg_col_bl_size=3.1
level=11,  2048
    avg_row_bl_size=1.7, avg_col_bl_size=1.6
level=12,  3200
    avg_row_bl_size=1.1, avg_col_bl_size=1.0


#### Rank allocation for given `hpart` and `ranks`

In [14]:
ra_losses, epochs, ranks_history = hat_A.rank_alloc(A, ranks, hat_A.hpart, freq=1)

itr=0, t=3, losses[0]=0.538446125505475, losses[-1]=0.4524837697549222, [2 2 2 2 2 2 2 1 1 1 1 1 1]
itr=1, t=6, 0.4166171853585573, [3 2 2 2 2 2 2 1 1 1 1 1 0]
itr=2, t=9, 0.3895054615469889, [4 2 2 2 2 2 2 1 1 1 1 0 0]
itr=3, t=12, 0.36527912567017645, [5 2 2 2 2 2 2 1 1 1 0 0 0]
itr=4, t=15, 0.3453470168491608, [6 2 2 2 2 2 2 1 1 0 0 0 0]
itr=5, t=18, 0.3259628881392974, [6 3 2 2 2 2 2 1 0 0 0 0 0]
itr=6, t=21, 0.3111795781464511, [7 3 2 2 2 2 2 0 0 0 0 0 0]
itr=7, t=24, 0.2999044888573783, [8 3 2 2 2 2 1 0 0 0 0 0 0]
itr=8, t=27, 0.29357581678318745, [9 3 2 2 2 1 1 0 0 0 0 0 0]
itr=9, t=30, 0.28923157389234744, [8 3 3 2 2 1 1 0 0 0 0 0 0]
itr=10, t=33, 0.28859675031784665, [7 4 3 2 2 1 1 0 0 0 0 0 0]
itr=11, t=36, 0.2866544676241888, [8 3 3 2 2 1 1 0 0 0 0 0 0]
quit: new rank allocation is worse
itr=12, t=39, 0.2857668762297508, [8 3 3 2 2 1 1 0 0 0 0 0 0]


# Comparison

In [15]:
print(f"{lr_losses[-1]=}")
print(f"{ff_losses[-1]=}")
print(f"{ra_losses[-1]=}")

lr_losses[-1]=0.5087598386871721
ff_losses[-1]=0.7105408720783372
ra_losses[-1]=0.2857668762297508


# Matvec operation $\hat A x$

In [16]:
hat_A.construct_sparse_format()
hat_A_val = hat_A.matrix()
x = np.random.randn(A.shape[1], 1)

In [17]:
np.allclose(hat_A.matvec(x), hat_A_val@x)

True

In [18]:
sp_flops = 2*rank*(n + m) - rank - m
print("flops ratio", m * ( 2*n - 1) * 1.0 / sp_flops)

flops ratio 84.68126134301271


In [19]:
%timeit hat_A.matvec(x)
%timeit np.matmul(hat_A_val, x)

260 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.91 ms ± 78.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
